# Load libraries and summary functions

In [2]:
#Reference http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/

In [3]:
#In this example we use cv to find the best base model then fit that model to the whole training set to make predictions 
#on the test set (This is what's in the link above)

In [4]:
suppressMessages({
library(dplyr) # for data manipulation
library(caret) # for model-building
library(pROC) # for AUC calculations
library(PRROC) # for Precision-Recall curve calculations
library(magrittr) # for pipes
library(caretEnsemble) #for stacking
})

In [5]:
options(repr.matrix.max.cols=50)
options(repr.matrix.max.rows=50)

In [6]:
set.seed(45406)

In [7]:
#Class1 == 0
#Calss2 == 1

aucSummary <- function(data, lev = NULL, model = NULL){
  
  index_class2 <- data$obs == "Class2"
  index_class1 <- data$obs == "Class1"
  
  #calc the auc metrics
  pr <- pr.curve(data$Class2[index_class2],
                 data$Class2[index_class1],
                 curve = FALSE)
  
  roc <- roc.curve(data$Class2[index_class2],
                   data$Class2[index_class1],
                   curve = FALSE)
  
  pr_out <- pr$auc.integral
  
  roc_out <- roc$auc
  
  obs <- data$obs
  
  
  #Assign threshold
  pre <- ifelse(data$Class2 > .5, 'Class2', 'Class1')
  
  #Calculate Accuracy  
  acc <- mean(obs == pre)
  
  out <- c(pr_out,roc_out,acc)  
  
  names(out) <-c("AUPRC","AUROC","Accuracy")
  
  out  
}

# Create test/train data sets

In [8]:
#Create data set
data_set <- twoClassSim(1000,
                    intercept = -6,
                    linearVars = 8,
                    noiseVars = 4)

In [9]:
#Create train/tests sets
index <- createDataPartition(data_set$Class, p = 6/10, list=FALSE)

train_set <- data_set[index,]
test_set <- data_set[-index,]

In [10]:
#Look at dim
train_set %>% dim()

train_set %>% head()

[1] 601  18

,TwoFactor1,TwoFactor2,Linear1,Linear2,Linear3,Linear4,Linear5,Linear6,Linear7,Linear8,Nonlinear1,Nonlinear2,Nonlinear3,Noise1,Noise2,Noise3,Noise4,Class
2,0.8097223,1.47437548,-0.8076689,-1.1319747,-0.2866550,1.78247610,0.16440952,0.26069540,0.5582444,-0.8972071,-0.04248304,0.9344437,0.91795414,0.8458606,1.7498821,1.6652243,0.5959273,Class1
3,1.2387178,0.01599239,-1.2053774,-1.0802625,2.2725103,-0.31690291,1.07191422,-0.52142051,1.0209103,0.8011080,-0.68616298,0.3735412,0.60716965,-0.6950674,-0.1531351,-0.1616989,1.7063336,Class1
5,-0.5131204,0.25986890,1.5314869,-0.4635079,1.5407780,-1.36485763,-2.01751135,-0.04614105,0.2751114,0.4361813,0.44862104,0.8372881,0.91693135,-0.1913738,0.6197387,-1.2638430,-0.1050231,Class2
6,1.0278531,1.61567486,-0.1555739,0.4161480,-0.1254957,0.07546147,0.93962369,-1.52580671,-0.1069174,-0.7167698,-0.66104746,0.4615281,0.14162099,-1.3987289,0.8233420,0.5832635,1.0977854,Class2
7,1.3514164,0.15930289,-0.9060178,0.9092948,-0.0106088,-0.17534387,0.06520887,0.16230173,-0.9382551,-1.0432937,0.01796992,0.9273337,0.18118455,-0.2059794,0.3343527,-0.5753191,-0.9176470,Class1
8,-0.3416407,-3.37577238,0.5900949,2.0505353,0.2392311,0.53155128,-0.68023745,1.96639816,1.4230917,1.2681464,-0.95171187,0.8373439,0.05076276,1.6775790,-1.4540916,0.1381279,-0.2966907,Class1


# Caret for ensemble 

In [1]:
#This ensemble only uses the output from two base models. 
#(Not the output of the two base models + the original varaibles like a stacked model )

# https://www.machinelearningplus.com/machine-learning/caret-package/

In [13]:
#Control
trainControl <- trainControl(method="repeatedcv", 
                             number=10,
                             index = createFolds(train_set$Class, 5),
                             repeats=3,
                             savePredictions="all", 
                             classProbs=TRUE)

#Base Models
algorithmList <- c('rf', 'gbm')

#Fit Base Models
models <- caretList(Class ~ ., data=train_set,
                    trControl=trainControl,
                    methodList=algorithmList,
                    metric = "Accuracy") 

#Get Results
results <- resamples(models)
summary(results)


Call:
summary.resamples(object = results)

Models: rf, gbm 
Number of resamples: 5 

Accuracy 
         Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
rf  0.7796258 0.7837838 0.7879418 0.7903508 0.7937500 0.8066528    0
gbm 0.7833333 0.7941788 0.7962578 0.7974151 0.8045738 0.8087318    0

Kappa 
         Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
rf  0.5121891 0.5364027 0.5438839 0.5476153 0.5642806 0.5813203    0
gbm 0.5250328 0.5543087 0.5680960 0.5636982 0.5764179 0.5946357    0


In [14]:
#Use different control for ensemble
stackControl <- trainControl(method="repeatedcv", 
                             number=10, 
                             repeats=3,
                             savePredictions=TRUE, 
                             classProbs=TRUE)


#Fit ensemble 
stack.glm <- caretStack(models, method="glm", metric="Accuracy", trControl=stackControl)
print(stack.glm)

A glm ensemble of 2 base models: rf, gbm

Ensemble results:
Generalized Linear Model 

2404 samples
   2 predictor
   2 classes: 'Class1', 'Class2' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 3 times) 
Summary of sample sizes: 2163, 2163, 2163, 2163, 2165, 2164, ... 
Resampling results:

  Accuracy   Kappa    
  0.8071213  0.5899372



In [19]:
#summary
summary(stack.glm)


Call:
NULL

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4988  -0.6056  -0.3705   0.5762   2.4169  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -3.1084     0.1321 -23.535   <2e-16 ***
rf            3.6916     0.3832   9.633   <2e-16 ***
gbm           3.0369     0.2610  11.638   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3224.5  on 2403  degrees of freedom
Residual deviance: 2077.7  on 2401  degrees of freedom
AIC: 2083.7

Number of Fisher Scoring iterations: 5
